## → Chapter Five  
*The Magic of Open APIs and Open Databases*

🔺After many failed attempts with Chinese websites, I decided to see how many titles I could find using just open databases based in the U.S., and planned to manually fill in the rest.  
Even if only a fraction of the books were found, it would still save time.

🔺To my surprise, I realized the managers had misunderstood the processand accidentally mislead me. When they said they couldn’t retrieve Chinese book info by ISBN, what they really meant was that the data wouldn’t auto-populate in Shopify through a barcode scan—so they gave up and resorted to manual entry. It became clear that these resources had always been available, the people before me just didn’t know where or how to look.  


🔺But actually, there are several open database options that provide structured ISBN data and return a good range of results. Out of roughly 700 titles, **Zotero** (through WorldCat and other databases) returned ~200 results, **ISBNdb** returned ~520 results; and **isbnsearch.org** worked well for quick scraping but ran into a rate limit issue.

Below, I provide the tutorial to:   
- Use the **ISBNdb API** to retrieve detailed information on books by ISBNs
- Scrape **isbnsearch.org** using BeautifulSoup and retrieve basic book information by ISBN without the book descriptions
- Look up book info by ISBN in bulk through **Zotero**

---
## → 第五章  
*踏破铁鞋无觅处，开放数据在眼前*

🔺在多次尝试中文网站失败之后，我决定试试看，单纯依赖美国的开放数据库，能找到多少图书的信息，并计划把剩下找不到的部分手动补上。  
即使只能找回一小部分书目，也能节省不少时间。

🔺令我意外的是，我意识到店里的管理人员其实误解了整个流程，并因此误导了我。他们口中的“无法通过ISBN查到中文书籍的信息”，其实是指这些信息无法通过扫码自动出现在Shopify里。于是他们就放弃了自动化的方法，转而全靠人工输入。后来我才意识到，这些资源其实一直都在，只是之前没有人知道该去哪里找，或者怎么用。

🔺事实上，很多开放平台确实提供结构化的ISBN书目信息，并能返回大量结果。在我大约700本书的样本中：  
- **Zotero**（通过 WorldCat）返回了约 200 本  
- **ISBNdb** 返回了约 520 本  
- **isbnsearch.org** 则适合用 BeautifulSoup 快速爬取基础信息，但有请求次数限制  

🔺我首先使用了 **ISBNdb API**，它能稳定返回书籍的出版方、出版日期、分类标签、简介等字段。  
**Zotero** 允许通过它的文献管理功能批量查询ISBN。  
**isbnsearch.org** 则适合快速、轻量地爬取基础字段（不含书籍简介）。

下方是关于以下方法的详细教程：  
- 如何使用 **ISBNdb API** 通过 ISBN 获取详细图书信息  
- 如何使用 **BeautifulSoup** 爬取 **isbnsearch.org** 并提取基础字段（不含简介）  
- 如何通过 **Zotero** 批量查询 ISBN 并获取图书信息

---



### **→ I. Querying ISBNdb API for Book Information**  

**→ 1.❗Requirements:**  
🔺You must register on [ISBNdb.com](https://isbndb.com/) and purchase a plan to access the API.
🔺Once registered, you will receive a personal API key, which you'll use for authorization.

**→ 2. ISBNdb and API explained**  
🔺[ISBNdb](https://isbndb.com/) is a subscription-based API that provides structured metadata for books.
It supports queries by ISBN and is useful for bulk cataloging projects.

🔺An **API** (Application Programming Interface) is a tool that allows different software systems to talk to each other.  
In this case, we are sending a request from our Python script to **ISBNdb.com**, which is a website that stores metadata for books.  
In response, the API sends us structured data about the book we ask for, based on its **ISBN** (International Standard Book Number).

🔺Using the code below, I was able to:   
- Send each isbn in my list to the ISBNdb API using the format `https://api2.isbndb.com/book/{isbn}`  
- Saving the returned data into a structured csv

In [ ]:
import pandas as pd
import requests
import time

# define range and api key
start_idx = 234 # you can re-define the start and end ids to choose how many entries from the list you want to query at one time.
end_idx = 335
API_KEY = 'your_api_key_here'  # replace with your actual API key

if API_KEY == '':
    print('Error: you need to subscribe to ISBNdb.com and provide your API key.')

# set headers
HEADERS = {
    'accept': 'application/json',
    'Authorization': API_KEY,
    'Content-Type': 'application/json',
}

# function to query by isbn
def get_book_info(isbn):
    url = f'https://api2.isbndb.com/book/{isbn}'
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            return response.json().get("book", {})
        else:
            print(f"ISBN {isbn} not found or error ({response.status_code})")
            return {}
    except Exception as e:
        print(f"Error fetching ISBN {isbn}: {e}")
        return {}

# process isbn list from csv
def process_isbn_csv(csv_path, start_idx=0, end_idx=None):
    df = pd.read_csv(csv_path)
    if 'CODECONTENT' not in df.columns:
        raise ValueError("CSV must contain a 'CODECONTENT' column with ISBNs.")
    
    isbn_list = df['CODECONTENT'].astype(str).tolist()
    if end_idx is None:
        end_idx = len(isbn_list)

    results = []
    for idx, isbn in enumerate(isbn_list[start_idx:end_idx], start=start_idx):
        isbn = isbn.strip()
        print(f"[{idx}] Fetching: {isbn}")
        book_data = get_book_info(isbn)
        book_data['isbn_searched'] = isbn  
        results.append(book_data)
        time.sleep(1)  

    result_df = pd.json_normalize(results)
    return result_df

# run script (make sure the csv file path, start id, end id are all correct)
if __name__ == "__main__":
    result_df = process_isbn_csv('../data/scannedResults.csv', start_idx, end_idx)
    print(result_df.head())
    result_df.to_csv("isbndbResults.csv", index=False)

### **→ II. Scraping isbnsearch.org for book data by ISBNs**

**→ 1. Website Structure**  

🔺I used BeautifulSoup to scrape the isbnsearch.org website because it does not have any restrictions. I only introcuded a sleep time to be polite. 

🔺Much like of the process I used before with BeautifulSoup, I needed to find the right URL structure for the pages containing information for each book and extract the information from HTML. Scraping this site was even simpler because it has a simple URL structure that will lead me directly to the book info page given an ISBN. 

🔺with BS4 and request, I was able to perform the tasks listed below for the first 10 entries in my ISBN list:

- Use `requests` and `BeautifulSoup` to fetch and parse HTML
- Extract key fields from the `<div class="bookinfo">` section
- Save the results to a structured csv

🔺However, I encountered a reCAPTCHA after that. Even though the site's robots.txt allows all URLs on the site are allowed to be crawled, they still have a rate limiting in place. So I would recommend running the code as below with at least random sleep time and in small batches.

  → <img src="../images/0502.png" alt="Add Items by Identifier in Zotero" width="200px">



In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random

# path to CSV with scanned ISBNs
csv_path = '../data/scannedResults.csv'

# load ISBNs
df = pd.read_csv(csv_path)
isbn_list = df['CODECONTENT'][:100].astype(str).tolist()

# base URL
base_url = 'https://isbnsearch.org/isbn/'

# headers to mimic real browser
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

# store results
results = []

for idx, isbn in enumerate(isbn_list):
    url = base_url + isbn
    print(f"[{idx}] Scraping: {url}")

    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch ISBN {isbn} — status {response.status_code}")
            continue

        soup = BeautifulSoup(response.content, 'html.parser')
        info_div = soup.find('div', class_='bookinfo')

        if not info_div:
            print(f"No data found for ISBN {isbn}")
            continue

        data = {
            'isbn': isbn,
            'title': info_div.find('h1').get_text(strip=True) if info_div.find('h1') else None,
            'isbn_13': None,
            'isbn_10': None,
            'author': None,
            'edition': None,
            'binding': None,
            'publisher': None,
            'published': None,
        }

        # extract fields from <p> elements
        for p in info_div.find_all('p'):
            text = p.get_text(strip=True)
            if text.startswith('ISBN-13:'):
                data['isbn_13'] = p.text.replace('ISBN-13:', '').strip()
            elif text.startswith('ISBN-10:'):
                data['isbn_10'] = p.text.replace('ISBN-10:', '').strip()
            elif text.startswith('Author:'):
                data['author'] = p.text.replace('Author:', '').strip()
            elif text.startswith('Edition:'):
                data['edition'] = p.text.replace('Edition:', '').strip()
            elif text.startswith('Binding:'):
                data['binding'] = p.text.replace('Binding:', '').strip()
            elif text.startswith('Publisher:'):
                data['publisher'] = p.text.replace('Publisher:', '').strip()
            elif text.startswith('Published:'):
                data['published'] = p.text.replace('Published:', '').strip()

        results.append(data)

        # random sleep to reduce block risk
        time.sleep(random.uniform(5, 10))

    except Exception as e:
        print(f"Error scraping ISBN {isbn}: {e}")
        continue

# save to CSV
result_df = pd.DataFrame(results)
result_df.to_csv('../data/isbnsearchResults.csv', index=False, encoding='utf-8-sig')
print("✅ Data saved to isbnsearchResults.csv")


### **→III. Bulk ISBN Lookup in Zotero**

**→ 1. Downloading Zotero**

[Zotero](https://www.zotero.org/) is a free and open-source reference management tool used by researchers to collect, organize, and cite sources. It also allows you to add books by ISBN, DOI, or other identifiers and automatically pulls metadata from a number of databases. For example, many of the books I retrieved via bulk ISBN import were matched through **WorldCat**.
You can [download Zotero on the official website](https://www.zotero.org/download/). 

**→2. Combine ISBNs into a single string to query in Zotero**
Zotero allows bulk look up of ISBNs, but I needed to put all the values in a single comma separated string in Python:

```python
import pandas as pd

df = pd.read_csv('../data/scannedResults.csv')
isbn_string = ",".join(df['CODECONTENT'].dropna().astype(str).tolist())
print(isbn_string)
```
I then copied the string into the input box for "Add Items by Identifier" in Zotero, and wait for Zotero to process and load the results.

  → <img src="../images/0501.png" alt="Add Items by Identifier in Zotero" width="400px">


**→3. Export Library**

Once all books are successfully looked up, I exported my library into a csv named **zoteroExport.csv** and saved it in the data folder.
  → <img src="../images/0503.png" alt="Export Library in Zotero" height="300px">




---
### **→ I. 使用 ISBNdb API 查询图书信息**

**→ 1.❗使用要求：**  
🔺你需要在 [ISBNdb.com](https://isbndb.com/) 注册账户，并购买一个 API 订阅计划。  
🔺注册后，你会获得一个专属的 API 密钥，用于授权访问接口。

**→ 2. 什么是 ISBNdb 和 API？**  
🔺[ISBNdb](https://isbndb.com/) 是一个基于订阅的 API 服务，提供结构化的图书元数据。  
它支持通过 ISBN 查询图书信息，适合用于批量图书目录整理等项目。

🔺**API**（应用程序接口）是一种让不同软件系统之间进行通信的工具。  
在这个案例中，我们通过 Python 脚本向 **ISBNdb.com** 发送请求，它是一个图书元数据的数据库网站。  
API 会根据我们提供的 **ISBN（国际标准书号）**，返回对应图书的结构化信息。

🔺通过下面的代码，我实现了以下操作：  
- 将我列表中的每个 ISBN 发送至 ISBNdb API，使用的地址格式为 `https://api2.isbndb.com/book/{isbn}`  
- 将返回的数据保存为结构化的 CSV 文件


In [ ]:
import pandas as pd
import requests
import time

# define range and api key
start_idx = 234 # you can re-define the start and end ids to choose how many entries from the list you want to query at one time.
end_idx = 235
API_KEY = 'your_api_key_here'  # replace with your actual API key

if API_KEY == '':
    print('Error: you need to subscribe to ISBNdb.com and provide your API key.')

# set headers
HEADERS = {
    'accept': 'application/json',
    'Authorization': API_KEY,
    'Content-Type': 'application/json',
}

# function to query a single isbn
def get_book_info(isbn):
    url = f'https://api2.isbndb.com/book/{isbn}'
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 200:
            return response.json().get("book", {})
        else:
            print(f"ISBN {isbn} not found or error ({response.status_code})")
            return {}
    except Exception as e:
        print(f"Error fetching ISBN {isbn}: {e}")
        return {}

# process isbn list from csv
def process_isbn_csv(csv_path, start_idx=0, end_idx=None):
    df = pd.read_csv(csv_path)
    if 'CODECONTENT' not in df.columns:
        raise ValueError("CSV must contain a 'CODECONTENT' column with ISBNs.")
    
    isbn_list = df['CODECONTENT'].astype(str).tolist()
    if end_idx is None:
        end_idx = len(isbn_list)

    results = []
    for idx, isbn in enumerate(isbn_list[start_idx:end_idx], start=start_idx):
        isbn = isbn.strip()
        print(f"[{idx}] Fetching: {isbn}")
        book_data = get_book_info(isbn)
        book_data['isbn_searched'] = isbn  
        results.append(book_data)
        time.sleep(1)  # to respect API rate limits

    result_df = pd.json_normalize(results)
    return result_df

# run script (make sure the csv file path, start id, end id are all correct)
if __name__ == "__main__":
    result_df = process_isbn_csv('../data/scannedResults.csv', start_idx, end_idx)
    print(result_df.head())
    result_df.to_csv("isbndbResults.csv", index=False)

### **→ II. 使用 isbnsearch.org 通过 ISBN 爬取图书数据**

**→ 1. 网站结构**

🔺我使用 BeautifulSoup 爬取了 isbnsearch.org 网站的数据，因为该网站没有明显的反爬虫机制。我只添加了延时（sleep time）以示礼貌。  

🔺就像我之前使用 BeautifulSoup 的流程一样，我需要找到每本书信息页面的正确 URL 结构，并从 HTML 中提取信息。  
这个网站的结构更加简单，直接通过 ISBN 构建 URL 即可跳转到对应的图书信息页面。

🔺使用 `requests` 和 `BeautifulSoup`，我完成了以下任务（以我的 ISBN 列表中前 10 个条目为例）：

- 使用 `requests` 获取网页 HTML 内容，并用 `BeautifulSoup` 解析  
- 从 `<div class="bookinfo">` 区块中提取关键字段  
- 将提取的数据保存为结构化的 CSV 文件  

🔺不过，在运行了约 10 次请求之后，我遇到了 **reCAPTCHA 验证**。虽然该网站的 `robots.txt` 文件允许爬取所有路径，但它们仍然设置了访问频率限制。

→ <img src="../images/0502.png" alt="Add Items by Identifier in Zotero" width="200px">


因此，我建议在运行下面的代码时，务必添加 **随机延时**，并将请求拆分成 **小批次**运行，以避免被封锁


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random

# path to CSV with scanned ISBNs
csv_path = '../data/scannedResults.csv'

# load ISBNs
df = pd.read_csv(csv_path)
isbn_list = df['CODECONTENT'][:100].astype(str).tolist()

# base URL
base_url = 'https://isbnsearch.org/isbn/'

# headers to mimic real browser
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

# store results
results = []

for idx, isbn in enumerate(isbn_list):
    url = base_url + isbn
    print(f"[{idx}] Scraping: {url}")

    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        if response.status_code != 200:
            print(f"Failed to fetch ISBN {isbn} — status {response.status_code}")
            continue

        soup = BeautifulSoup(response.content, 'html.parser')
        info_div = soup.find('div', class_='bookinfo')

        if not info_div:
            print(f"No data found for ISBN {isbn}")
            continue

        data = {
            'isbn': isbn,
            'title': info_div.find('h1').get_text(strip=True) if info_div.find('h1') else None,
            'isbn_13': None,
            'isbn_10': None,
            'author': None,
            'edition': None,
            'binding': None,
            'publisher': None,
            'published': None,
        }

        # extract fields from <p> elements
        for p in info_div.find_all('p'):
            text = p.get_text(strip=True)
            if text.startswith('ISBN-13:'):
                data['isbn_13'] = p.text.replace('ISBN-13:', '').strip()
            elif text.startswith('ISBN-10:'):
                data['isbn_10'] = p.text.replace('ISBN-10:', '').strip()
            elif text.startswith('Author:'):
                data['author'] = p.text.replace('Author:', '').strip()
            elif text.startswith('Edition:'):
                data['edition'] = p.text.replace('Edition:', '').strip()
            elif text.startswith('Binding:'):
                data['binding'] = p.text.replace('Binding:', '').strip()
            elif text.startswith('Publisher:'):
                data['publisher'] = p.text.replace('Publisher:', '').strip()
            elif text.startswith('Published:'):
                data['published'] = p.text.replace('Published:', '').strip()

        results.append(data)

        # random sleep to reduce block risk
        time.sleep(random.uniform(5, 10))

    except Exception as e:
        print(f"Error scraping ISBN {isbn}: {e}")
        continue

# save to CSV
result_df = pd.DataFrame(results)
result_df.to_csv('../data/isbnsearchResults.csv', index=False, encoding='utf-8-sig')
print("✅ Data saved to isbnsearchResults.csv")


### **→ III. 使用 Zotero 批量查询 ISBN 图书信息**

**→ 1. 下载 Zotero**

[Zotero](https://www.zotero.org/) 是一个免费开源的参考文献管理工具，被广泛用于收集、整理和引用各类文献资料。它支持通过 ISBN、DOI 或其他识别码添加图书，并可自动从多个数据库中抓取元数据。  
例如，我通过批量导入 ISBN 查询时，Zotero 很多结果都来自于 **WorldCat** 数据库。  
你可以在 [Zotero 官方网站下载](https://www.zotero.org/download/) 桌面客户端。

---

**→ 2. 将 ISBN 合并为一个字符串以便查询**

Zotero 支持批量 ISBN 查询，但需要将所有 ISBN 合并为一个逗号分隔的字符串。我使用以下 Python 代码完成这一步：

```python
import pandas as pd

df = pd.read_csv('../data/scannedResults.csv')
isbn_string = ",".join(df['CODECONTENT'].dropna().astype(str).tolist())
print(isbn_string)
```
复制输出的字符串，在 Zotero 中点击上方工具栏的“Add Items by Identifier”，将字符串粘贴进输入框，然后按下 Enter，等待 Zotero 逐个查询并加载元数据。

  → <img src="../images/0501.png" alt="Add Items by Identifier in Zotero" width="400px">

**→ 3. 导出图书库为 CSV**

当所有图书信息都成功查找到后，我将图书库导出为名为 **zoteroExport.csv** 的 CSV 文件，并将其保存在 `data` 文件夹中。  
  → <img src="../images/0503.png" alt="在 Zotero 中导出图书库" height="300px">